In [10]:
import os
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import (Conv2D, ZeroPadding2D)
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import InputLayer
from tensorflow_core.python.keras.layers import  MaxPooling2D, Softmax
from tqdm.notebook import tqdm

In [2]:
def make_conv(in_size, out_size, **params):
    assert len(params) == 2, "Only kernel_size and filters are allowed in **params"
    kernel_size = params['kernel_size']
    filters = params['filters']
    w_i = in_size
    w_o = out_size

    tp = w_o - w_i - 1 + kernel_size
    p1 = tp // 2
    p2 = tp - p1
    assert w_o == w_i + 1 - kernel_size + p1 + p2
    block = [
        Conv2D(filters, kernel_size, ),
        ZeroPadding2D(((p1, p2), (p1, p2)))
    ]
    return block

def create_model(in_size, kernel_size, initial_filters):
    model = Sequential([
        InputLayer(input_shape=(in_size, in_size, 3)),
        *make_conv(in_size, out_size=in_size, filters=initial_filters, kernel_size=kernel_size),
        MaxPooling2D(pool_size=(2,2)),
        *make_conv(in_size//2, out_size = in_size//2, filters=initial_filters*2, kernel_size=kernel_size),
        MaxPooling2D(pool_size=(2,2)),
        *make_conv(in_size//4, out_size = in_size//4, filters=initial_filters*4, kernel_size=kernel_size),
        MaxPooling2D(pool_size=(2,2)),
        *make_conv(in_size//8, out_size = in_size//8, filters=initial_filters*8, kernel_size=kernel_size),
        MaxPooling2D(pool_size=(2,2)),
        Flatten(),
        Softmax()
    ])

    return model


In [12]:
models = {}
for kernel_size in  [1, 2, 3, 4, 5, 6, 7, 8, 9]:
    name = f'kernel_size_{kernel_size}_{kernel_size}'
    model = create_model(in_size=256, initial_filters=8, kernel_size=kernel_size)
    models[name] = model

In [13]:
# Print models
for name, model in models.items():
    print('-'*80)
    print('NAME ' + name)
    model.summary()


--------------------------------------------------------------------------------
NAME kernel_size_1_1
Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_32 (Conv2D)           (None, 256, 256, 8)       32        
_________________________________________________________________
zero_padding2d_32 (ZeroPaddi (None, 256, 256, 8)       0         
_________________________________________________________________
max_pooling2d_32 (MaxPooling (None, 128, 128, 8)       0         
_________________________________________________________________
conv2d_33 (Conv2D)           (None, 128, 128, 16)      144       
_________________________________________________________________
zero_padding2d_33 (ZeroPaddi (None, 128, 128, 16)      0         
_________________________________________________________________
max_pooling2d_33 (MaxPooling (None, 64, 64, 16)        0         
__________________

In [14]:
# Convert the models
output_dir = 'tflite_models/kernels'
os.makedirs(output_dir, exist_ok=True)

for name, model in tqdm(models.items()):
    converter = tf.lite.TFLiteConverter.from_keras_model(model)
    tflite_model = converter.convert()

    # write it to file
    filepath = os.path.join(output_dir, name + '.tflite')
    with open(filepath, 'wb') as f:
        f.write(tflite_model)